# Jupyter Notebook & Spark

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Налаживаем-взаимодействие-Jupyter-Notebook-и-Spark" data-toc-modified-id="Налаживаем-взаимодействие-Jupyter-Notebook-и-Spark-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Налаживаем взаимодействие Jupyter Notebook и Spark</a></span></li><li><span><a href="#Пробуем-Spark-в-деле:-считаем-M&amp;Ms" data-toc-modified-id="Пробуем-Spark-в-деле:-считаем-M&amp;Ms-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Пробуем Spark в деле: считаем M&amp;Ms</a></span></li></ul></div>

## Налаживаем взаимодействие Jupyter Notebook и Spark

Действуем по статье: [Get Started with PySpark and Jupyter Notebook in 3 Minutes](https://medium.com/sicara/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f#:~:text=There%20are%20two%20ways%20to%20get%20PySpark%20available,Jupyter%20Notebook%20and%20load%20PySpark%20using%20findSpark%20package)

Загружаем findspark (делается один раз, затем можно закомментировать):

In [1]:
#pip install findspark

Устанавливаем необходимые библиотеки:

In [2]:
import findspark
findspark.init()
import pyspark
import random

Запускаем небольшую программу из статьи:

In [3]:
sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

3.1415942


## Пробуем Spark в деле: считаем M&Ms 

Теперь опробуем программу подсчёта M&Ms. Импортируем библиотеки:

In [6]:
from pyspark.sql import SparkSession

Задаём Спарк-сессию:

In [7]:
spark = (SparkSession
        .builder
        .appName("PythonMnMCount")
        .getOrCreate())

Задаём путь к файлу, читаем его и выводим 5 строк:

In [8]:
# get the M&M data set file name
mnm_file = '/C:/datasets/mnm_dataset.csv'

# read the file into a Spark DataFrame
mnm_df = (spark.read.format("csv")
        .option("header", "true")
        .option("inferSchema", "true")
        .load(mnm_file))

# show 5 rows
mnm_df.show(n=5, truncate=False)

+-----+------+-----+
|State|Color |Count|
+-----+------+-----+
|TX   |Red   |20   |
|NV   |Blue  |66   |
|CO   |Blue  |79   |
|OR   |Blue  |71   |
|WA   |Yellow|93   |
+-----+------+-----+
only showing top 5 rows



Делаем трансофрмацию исходного файла и выводим 60 строк - штаты и цвета M&Ms с наибольшим количеством:

In [9]:
# aggregate count of all colors and groupBy state and color
# orderBy descending order
count_mnm_df = (mnm_df.select("State", "Color", "Count")
                    .groupBy("State", "Color")
                    .sum("Count")
                    .orderBy("sum(Count)", ascending=False))

# show all the resulting aggregation for all the dates and colors
count_mnm_df.show(n=60, truncate=False)

+-----+------+----------+
|State|Color |sum(Count)|
+-----+------+----------+
|CA   |Yellow|100956    |
|WA   |Green |96486     |
|CA   |Brown |95762     |
|TX   |Green |95753     |
|TX   |Red   |95404     |
|CO   |Yellow|95038     |
|NM   |Red   |94699     |
|OR   |Orange|94514     |
|WY   |Green |94339     |
|NV   |Orange|93929     |
|TX   |Yellow|93819     |
|CO   |Green |93724     |
|CO   |Brown |93692     |
|CA   |Green |93505     |
|NM   |Brown |93447     |
|CO   |Blue  |93412     |
|WA   |Red   |93332     |
|WA   |Brown |93082     |
|WA   |Yellow|92920     |
|NM   |Yellow|92747     |
|NV   |Brown |92478     |
|TX   |Orange|92315     |
|AZ   |Brown |92287     |
|AZ   |Green |91882     |
|WY   |Red   |91768     |
|AZ   |Orange|91684     |
|CA   |Red   |91527     |
|WA   |Orange|91521     |
|NV   |Yellow|91390     |
|UT   |Orange|91341     |
|NV   |Green |91331     |
|NM   |Orange|91251     |
|NM   |Green |91160     |
|WY   |Blue  |91002     |
|UT   |Red   |90995     |
|CO   |Orang

Убеждаемся, что у нас именно 60 строк:

In [12]:
print("Total Rows = %d" % (count_mnm_df.count()))

Total Rows = 60


И ещё раз трансформируем наш файл - выводим данные только для штата Калифорния:

In [11]:
# find the aggregate count for California by filtering
ca_count_mnm_df = (mnm_df.select("*")
                       .where(mnm_df.State == 'CA')
                       .groupBy("State", "Color")
                       .sum("Count")
                       .orderBy("sum(Count)", ascending=False))

# show the resulting aggregation for California
ca_count_mnm_df.show(n=10, truncate=False)

+-----+------+----------+
|State|Color |sum(Count)|
+-----+------+----------+
|CA   |Yellow|100956    |
|CA   |Brown |95762     |
|CA   |Green |93505     |
|CA   |Red   |91527     |
|CA   |Orange|90311     |
|CA   |Blue  |89123     |
+-----+------+----------+



Готово! Spark работает, и его удобно запускать через Jupyter Notebook (особенно если вы выпускник Яндекс Практикума 😉)